THIS IS A CODE TO CREATE CSV FOR MEDIS ATTACK FINAL OUTPUT (ATTACK SUCCESS RATE, AVERAGE DISTORTION) WITH AND WITHOUT PARALLEL FUSION IN TARGETED ATTACK SETTINGS.

In [ ]:
import os
os.chdir('/content/drive/MyDrive/')

In [ ]:
# IMPORT PACKAGES

import pandas as pd
import numpy as np
import csv
import torch
import copy
import sys
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# FUNCTION TO GET DATA FROM SAVED CSV FILE

def get_data_from_csv(folder,dataset,loss,target):
  if dataset is None:
    fileName = '{0}/{1}/target{2}_predTarget.csv'.format(folder,loss,target)
  else:
    fileName = '{0}/{1}/{2}/target{3}_predTarget.csv'.format(folder,dataset,loss,target)
  df = pd.read_csv(fileName,index_col='Input Image')
  cols = df.columns # column
  rows = df.index   # row
  for i in range(2,len(cols)): # replace 'Nan' values with '0' in csv dataframe
    df[cols[i]]  = df[cols[i]].fillna(0)
  return df,cols,rows


In [ ]:
def ModificationForIoU(dataFrame,columns):
  miou_columns = ['MeanIoU','MeanIoU.1','MeanIoU.2','MeanIoU.3','MeanIoU.4','MeanIoU.5','MeanIoU.6','MeanIoU.7','MeanIoU.8','MeanIoU.9']
  for column in miou_columns:
    dataFrame[column] = dataFrame[column].apply(lambda x:x*2 if x != 1 else x)
    #print(dataFrame[column])
  return dataFrame

In [ ]:
# FUNCTION TO GET AVERAGE OF A LIST OF LIST

def get_avg(y):
  step_avg = []
  miou_avg = []
  final_val = []
 # if failed_cases + len(y) == len(rows): # check....failed case + success case = total no. of images
  for a in range(len(y)): # loop for all success cases
    step_avg.append(y[a][1])
    miou_avg.append(y[a][2])
  metrics = [step_avg,miou_avg]
  for m in metrics:
    final_val.append(np.round(np.average(m),3))
  return final_val


In [ ]:
# FUNCTION TO GET INITIAL RESULT

def get_result(df,columns,rows,l_inf,threshold,parallel_fusion):
  y = []
  success_cases = 0
  for row in rows: # loop for each row
    x = []
    r = df.loc[row] # get complete row information(entries)
    x.append(row)
    for i in range(1,l_inf): # loop for all steps upto l_inf
      # condition for first step
      if (r['MeanIoU'] >= threshold): # check if mean iou is greater than threshold
        x.extend([r['Step'],r['MeanIoU']])  # only for mean iou and dice
        success_cases += 1
        y.append(x)
        break
      # condition for other steps
      elif (r['Step.'+str(i)] < l_inf+1) and ((r['MeanIoU.'+str(i)] >= threshold)):
        x.extend([r['Step.'+str(i)],r['MeanIoU.'+str(i)]])
        success_cases += 1
        y.append(x)
        break

  if parallel_fusion:
    return y,success_cases

  else:
    final_val = get_avg(y)
    return final_val,success_cases


In [ ]:
# FUNCTION TO GET RESULT WITHOUT PARALLEL FUSION

def calculate_metrics_without_parellelFusion(folder,dataset,l_inf,threshold,parallel_fusion):
  losses = ['Log_Cosh_Dice_Loss','BCE_Dice_Loss', 'Exponential_Log_Loss', 'wBCE_wIoU_Loss', 'Lovasz_Hinge_Loss','Combined_Loss']
  loss_result = []
  for loss in losses:
    if dataset == None:
      target_result = ['Skin_Lesion', loss]
    else:
      target_result = [dataset,loss]
    for target in range(1,5):
      dataFrame,columns,rows = get_data_from_csv(folder,dataset,loss,target) # get csv
      if target == 3 or target == 4: # change mIoU to IoU for target 3 and 4
        dataFrame = ModificationForIoU(dataFrame,columns)
      final_val,success_count = get_result( dataFrame,columns,rows,l_inf,threshold,parallel_fusion)
      step, miou = final_val
      attack_success_rate = round(( success_count / len(rows) ) * 100, 2)
      target_result.extend([attack_success_rate,step]) # append values for each target
    loss_result.append(target_result) # append values for each loss
  final_result = []
  final_result.extend(loss_result) # get final list of list

  return final_result

In [ ]:
# FUNCTION TO GET RESULT WITH PARALLEL FUSION

def calculate_metrics_with_parellelFusion(folder,dataset,l_inf,threshold,parallel_fusion):
  losses = ['Log_Cosh_Dice_Loss','BCE_Dice_Loss', 'Exponential_Log_Loss', 'wBCE_wIoU_Loss', 'Lovasz_Hinge_Loss','Combined_Loss']
  if dataset == None:
    target_result = ['Skin_Lesion']
  else:
    target_result = [dataset]

  for target in range(1,5): # loop for each target
    all_images_info = {}
    for loss in losses: # loop for each loss
      dataFrame,columns,rows = get_data_from_csv(folder,dataset,loss,target) # get csv
      if target == 3 or target == 4: # change mIoU to IoU for target 3 and 4
        dataFrame = ModificationForIoU(dataFrame,columns)
      final_val,success_count = get_result(dataFrame,columns,rows,l_inf,threshold,parallel_fusion)
      all_images_info.update({loss:final_val}) # a dict with loss(str) as key and metrics(list of list) as values; length(dict) = no.of losses

    image_wise_all_loss_metrics = {}
    D = copy.deepcopy(all_images_info)
    for loss_name, value_list in D.items(): # loop for each key,value in all_images_info
      for idx, metrics_array in enumerate(value_list): # loop for each entry in value list
        image_name = metrics_array[0] # get image name
        metrics_array.insert(0, loss_name) # insert loss name as first entry in each list of value list
        if image_name not in image_wise_all_loss_metrics:
          metrics_array_without_name = copy.deepcopy(metrics_array)  # a list of all metrics with loss name at 0 and image name at 1 position
          del metrics_array_without_name[1] # remove image name from list
          image_wise_all_loss_metrics[image_name] = [metrics_array_without_name] # a dict with 'image_name' as key and 'metrics_array_without_name' as values
        else:
          metrics_array_without_name = copy.deepcopy(metrics_array)
          del metrics_array_without_name[1]
          image_wise_all_loss_metrics[image_name].append(metrics_array_without_name)  #

    avg_list = []
    for image_name,metric_list in image_wise_all_loss_metrics.items(): # loop for each entry in 'image_wise_all_loss_metrics'
      t = sorted(metric_list, key=lambda e: (e[1],-e[2])) # sorting the values based on least distortion and highest mIoU
      avg_list.append(t[0]) # append first entry to avg_list (list of list)

    Parellel_Fusion_final_result = get_avg(avg_list) # get avg of each list in avg_list
    step,miou = Parellel_Fusion_final_result
    attack_success_rate = round(( len(avg_list) / len(rows) ) * 100, 2)
    target_result.append(attack_success_rate) # append attack success rate
    target_result.append(step) # append step
  final_result = []
  final_result.extend([target_result])  # get final list of list

  return final_result

In [ ]:
# STARTS FROM HERE.......SET VARIABLES AND CALL FUNCTIONS......

l_inf = 10
threshold  = 0.45
parallel_fusion = True  # or False
result_folder = '/content/drive/MyDrive/Final_result_medis/' # folder where result will save
folder = ['/content/drive/MyDrive/PraNet/code_check_medis/attack_10steps_results/polyp/', '/content/drive/MyDrive/PraNet/code_check_medis/attack_10steps_results/skin_lesion/']
testdataset = ['CVC-300','CVC-ClinicDB','CVC-ColonDB','Kvasir']

if parallel_fusion: # get result if parallel fusion is True
  fileName = result_folder + 'with_pf_targeted.csv'   # filename for saved result
  Header1 = ['Dataset']
  Header2 = ['']
  for i in range(1,5):
    Header1.extend(['Target '+str(i),''])
    Header2.extend(['Attack Success Rate(%)','Distortion'])
  with open(fileName, 'w') as f:
    write = csv.writer(f) # using csv.writer method from CSV package
    write.writerow(Header1)
    write.writerow(Header2)
    for f in folder:
      if f == '/content/drive/MyDrive/PraNet/code_check_medis/attack_10steps_results/polyp/':
          for dataset in testdataset:
            final_result = calculate_metrics_with_parellelFusion(folder=f,dataset=dataset,l_inf=l_inf,threshold=threshold,parallel_fusion=parallel_fusion)
            write.writerows(final_result)  # save result in csv
      else:
        final_result = calculate_metrics_with_parellelFusion(folder=f,dataset=None,l_inf=l_inf,threshold=threshold,parallel_fusion=parallel_fusion)
        write.writerows(final_result) # save result in csv

    print('csv done for result with parallel fusion!!!')


else: # get result if parallel fusion is False
  fileName = result_folder + 'without_pf_targeted.csv' # filename for saved result
  Header1 = ['Dataset', 'Loss']
  Header2 = ['','']

  for i in range(1,5):
    Header1.extend(['Target '+str(i),''])
    Header2.extend(['Attack Success Rate(%)','Distortion'])
  with open(fileName, 'w') as f:
    write = csv.writer(f) # using csv.writer method from CSV package
    write.writerow(Header1)
    write.writerow(Header2)
    for f in folder:
      if f == '/content/drive/MyDrive/PraNet/code_check_medis/attack_10steps_results/polyp/':
        for dataset in testdataset:
          final_result = calculate_metrics_without_parellelFusion(folder=f,dataset=dataset,l_inf=l_inf,threshold=threshold,parallel_fusion=parallel_fusion)
          write.writerows(final_result) # save result in csv
      else:
        final_result = calculate_metrics_without_parellelFusion(folder=f,dataset=None,l_inf=l_inf,threshold=threshold,parallel_fusion=parallel_fusion)
        write.writerows(final_result) # save result in csv

  print('csv done for result without parallel fusion!!!')
